<br>
<br>
<br>
<h1><center>Assignment 03</center></h1>
<h2><center>Exploratory Data Analysis</center></h2>
<h4><center>[ISYS2047] Information Systems Solutions and Design</center></h4>
<h3><center>Hyper Parameter Tuning</center></h3>
<br>
<div style="text-align: right"><h4>Student name: Juyeon Kim</h4></div>
<div style="text-align: right"><h4>Student ID: S3741327</h4></div>
<br></br>

## Import libraries

In [1]:
# Library for pickling
import joblib

# Miscellabeous libraries
import numpy as np
import pandas as pd
#import collections
#import time

# Model libraries
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

# Library for performing grid search
from sklearn.model_selection import GridSearchCV

# Metrics library
#from sklearn.metrics import accuracy_score
#from sklearn.metrics import confusion_matrix
#from sklearn.metrics import recall_score
#from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
#from sklearn.metrics import classification_report
#from sklearn.metrics import roc_curve
#from sklearn.metrics import auc

# 1. Load the pickled training and testing sets

## 1.1. Load dataset for KNN

In [2]:
# Load the pickled files
X_train_knn = joblib.load('X_train-knn.pkl')  
y_train_knn = joblib.load('y_train-knn.pkl')
X_test_knn = joblib.load('X_test-knn.pkl')  
y_test_knn = joblib.load('y_test-knn.pkl')

# Check
print("train data:", X_train_knn.shape, y_train_knn.shape)
print("test data:", X_test_knn.shape, y_test_knn.shape)

train data: (4800, 11) (4800,)
test data: (1200, 11) (1200,)


In [3]:
X_train_knn.head()

,yrs_experience,family_size,education_level,income,fixed_deposit_acct,bpc_mg_low,bpc_mg_moderate,bpc_mg_no,bpc_cs_high,bpc_cs_low,bpc_cs_medium
2973,-0.819489,1.0,0.0,-1.548229,0.0,0,0,1,0,1,0
2774,1.536079,4.0,0.0,1.383690,0.0,0,0,1,0,1,0
3281,1.451951,2.0,1.0,-0.703837,0.0,0,0,1,0,1,0
2512,-0.735361,1.0,0.0,0.210922,0.0,0,0,1,0,1,0
457,0.947187,4.0,2.0,-0.352006,0.0,0,0,1,0,1,0


## 1.2. Load dataset for Random Forest

In [4]:
# Load the pickled files
X_train_rf = joblib.load('X_train-rf.pkl')  
y_train_rf = joblib.load('y_train-rf.pkl')
X_test_rf = joblib.load('X_test-rf.pkl')  
y_test_rf = joblib.load('y_test-rf.pkl')

# Check
print("train data:", X_train_rf.shape, y_train_rf.shape)
print("test data:", X_test_rf.shape, y_test_rf.shape)

train data: (4800, 13) (4800,)
test data: (1200, 13) (1200,)


In [5]:
X_train_rf.head()

,age,yrs_experience,family_size,education_level,income,share_trading_acct,fixed_deposit_acct,bpc_mg_low,bpc_mg_moderate,bpc_mg_no,bpc_cs_high,bpc_cs_low,bpc_cs_medium
2973,63.0,11.0,1.0,0.0,19.0,0.0,0.0,0,0,1,0,1,0
2774,35.0,39.0,4.0,0.0,144.0,0.0,0.0,0,0,1,0,1,0
3281,34.0,38.0,2.0,1.0,55.0,0.0,0.0,0,0,1,0,1,0
2512,62.0,12.0,1.0,0.0,94.0,0.0,0.0,0,0,1,0,1,0
457,57.0,32.0,4.0,2.0,70.0,0.0,0.0,0,0,1,0,1,0


# 2. Optimization
> ***Result***
> - KNeighborsClassifier will find the best set of parameters to improve model performance. 

## 2.1. Optimization for KNN

In [6]:
# List of relevant params
#     n_neighbors:int, default=5
#     weights: {‘uniform’, ‘distance’} or callable, default=’uniform’
#     algorithm: {‘auto’, ‘ball_tree’, ‘kd_tree’, ‘brute’}, default=’auto’

param_grid = [
    {
     "n_neighbors": range(1, 50), 
     "weights": ["uniform", "distance"], 
     "algorithm": ["auto", "ball_tree", "kd_tree", "brute"]
    }
]

# Instantiate a grid search object and fit it to the training data
clf = KNeighborsClassifier()
grid = GridSearchCV(clf, param_grid, verbose=0, cv=5, scoring='f1')
grid.fit(X_train_knn, y_train_knn)
print(f"Best params: {grid.best_params_}, score: {grid.best_score_:0.2f}")

Best params: {'algorithm': 'ball_tree', 'n_neighbors': 20, 'weights': 'distance'}, score: 0.67


## For Random Forest

- Please note running below code takes approx 3 hours

In [7]:
# List of relevant params
#     n_estimators: int, default=100
#     criterion: {“gini”, “entropy”}, default=”gini”
#     max_depth: int, default=None
#     min_samples_split: int or float, default=2
#     min_samples_leaf: int or float, default=1
#     min_weight_fraction_leaf: float, default=0.0
#     max_features: {“auto”, “sqrt”, “log2”}, int or float, default=”auto”
#     max_leaf_nodes: int, default=None

param_grid = [
    {
        'max_depth': np.arange(1,10), # added
        'min_samples_leaf': [1, 2, 5, 7, 10,15,20], # added
        'max_leaf_nodes' : [5, 10,15,20,25,30], # added
        "criterion": ["gini", "entropy"], 
        "max_features": ["auto", "sqrt", "log2"]
    }
]

# Instantiate a grid search object and fit it to the training data
clf = RandomForestClassifier()
grid = GridSearchCV(clf, param_grid, verbose=0, cv=5, scoring='f1')
grid.fit(X_train_rf, y_train_rf)
print(f"Best params: {grid.best_params_}, score: {grid.best_score_:0.2f}")

Best params: {'criterion': 'gini', 'max_depth': 8, 'max_features': 'sqrt', 'max_leaf_nodes': 30, 'min_samples_leaf': 1}, score: 0.70
